# Web Scraping

The following code is WIP and was meant to be used with sentimental analysis to understand the public sentiments to the FAA's approval of the unleaded fuel. Only some scrapes were completed; the sentiment analysis was not finished.

## Forum Scrapes

Based on this guide:
\
https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/

The forum scrape uses the Beautiful Soup library.

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

from bs4 import BeautifulSoup
import requests
import pandas as pd

## Pilots of America thread: "G100UL unleaded avgas approved"

In [2]:
link = 'https://www.pilotsofamerica.com/community/threads/g100ul-unleaded-avgas-approved.133398/'
page = 1
article = []
message = []

# Put the total number of pages for the discussion + 1
while page != 9:
    url = f'{link}page-{page}'
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    posts = soup.find_all('li', {'class': 'message'})
    for blockquote in soup.find_all('blockquote', class_='messageText'):
        message.append(blockquote.get_text(strip=True))
        article.append(link)
    page = page + 1


forum_df1 = pd.DataFrame({'discussion': article, 'posts': message})
forum_df1.posts = forum_df1.posts.str.strip() #Gets rid of whitespace in front and back
forum_df1.posts = forum_df1.posts.replace(r'\n',' ', regex=True)  #Makes everything one line

forum_df1

,discussion,posts
0,https://www.pilotsofamerica.com/community/thre...,In case you haven't heard yet...Yesterday Geor...
1,https://www.pilotsofamerica.com/community/thre...,"So, is this a drop in replacement, even for hi..."
2,https://www.pilotsofamerica.com/community/thre...,Jim K said:↑In case you haven't heard yet...Ye...
3,https://www.pilotsofamerica.com/community/thre...,"So, are these guys making money on the STCs th..."
4,https://www.pilotsofamerica.com/community/thre...,"dans2992 said:↑So, is this a drop in replaceme..."
...,...,...
276,https://www.pilotsofamerica.com/community/thre...,Doug Reid said:↑I am more concerned about the ...
277,https://www.pilotsofamerica.com/community/thre...,"ldhilljr13 said:↑I think many of us are, both ..."
278,https://www.pilotsofamerica.com/community/thre...,Andy Appeltans said:↑https://www.aopa.org/news...
279,https://www.pilotsofamerica.com/community/thre...,"Kevin Eggert said:↑As you said here, the artic..."


## Reddit Scrapes

Based on these guides:
\
https://www.geeksforgeeks.org/scraping-reddit-using-python/
\
https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

Creating PRAW (Python Reddit API Wrapper) instance to connect to Reddit. 
\
For privacy reasons, personal account information is removed.

In [3]:
!pip install praw

import praw
import pandas as pd
from praw.models import MoreComments

In [4]:
# Read-only instance
reddit_read_only = praw.Reddit(client_id = '',
                               client_secret = '',
                               user_agent = '')

# Values were removed - find out strings with your own Reddit app using the guides above

### Reddit thread 1: "FAA approves Unleaded fuel… FINALLY" from r/flying

In [5]:
url = 'https://www.reddit.com/r/flying/comments/osyqxi/faa_approves_unleaded_fuel_finally/'
submission = reddit_read_only.submission(url = url)

In [6]:
post_comments = []

submission.comments.replace_more(limit=0)
for comment in submission.comments.list():
    post_comments.append(comment.body)
    discussion = url

# creating a dataframe
reddit_df1 = pd.DataFrame(post_comments, columns = ['comment'])
reddit_df1

,comment
0,Glad my plane runs on mogas
1,Now if they could get their stupid ass medical...
2,What's the catch? Good news doesn't usually sn...
3,Does anyone/has anyone gotten lead blood tests...
4,Piper STC meow
...,...
256,That's not necessarily true about finding moto...
257,You're also not pushing two tons of steel with...
258,Those Japanese engineers like to build cool st...
259,"Today? As a major global corporation, obviousl..."


### Reddit thread 2: "FAA approves hundreds more engines to use unleaded avgas" from r/flying

In [7]:
url = 'https://www.reddit.com/r/flying/comments/qih2o1/faa_approves_hundreds_more_engines_to_use/'
submission = reddit_read_only.submission(url = url)

In [8]:
post_comments = []

submission.comments.replace_more(limit=0)
for comment in submission.comments.list():
    post_comments.append(comment.body)

# creating a dataframe
reddit_df2 = pd.DataFrame(post_comments, columns = ['comment'])
reddit_df2

,comment
0,I think the original list only included some L...
1,No Lycoming IO-360-A3B6 :(
2,[deleted]
3,Notably absent are the Lycoming 540 and Contin...
4,Wow... this is long overdue.\n\nI understand f...
...,...
151,"Interesting, TIL! I had no idea lead itself wa..."
152,You pay $6.50 for avgas wow. No wonder they'r...
153,You literally said TEL is “literally” the only...
154,I though it was the exhaust that was the issue...
